In [131]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [132]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [133]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [276]:
X = torch.ones(6,8)
K = torch.tensor([[-1.0, 1.0]])
X[:,3] = 0

Y = corr2d(X, K)
X,Y #corr2d(X.T,K) # 说明卷积核能提取垂直特征，无法提取水平特征

(tensor([[1., 1., 1., 0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 1.]]),
 tensor([[ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.,  0.,  0.]]))

In [277]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
net = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(13):
    Y_hat = net(X)
    l = (Y_hat - Y) ** 2
    net.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    net.weight.data[:] -= lr * net.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 336.724
epoch 4, loss 25413.824
epoch 6, loss 1950835.000
epoch 8, loss 149757072.000
epoch 10, loss 11496190976.000
epoch 12, loss 882512101376.000


In [261]:
nn.Conv2d(1,1, kernel_size=(3, 2), bias=False).weight.shape,nn.Conv2d(2,1, kernel_size=(1, 2), bias=False).weight.shape,nn.Conv2d(3,1, kernel_size=(1, 2), bias=False).weight.shape

(torch.Size([1, 1, 3, 2]), torch.Size([1, 2, 1, 2]), torch.Size([1, 3, 1, 2]))

In [272]:
net(X) - Y

tensor([[[[-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332],
          [-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332],
          [-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332],
          [-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332],
          [-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332],
          [-0.0332, -0.0139,  0.0000,  0.0000,  0.0000, -0.0193, -0.0332]]]],
       grad_fn=<SubBackward0>)